In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torchsummary

In [ ]:
from tqdm import tqdm

In [ ]:
import PIL
from collections import defaultdict
import torch

In [ ]:
import torch.nn as nn
import torchvision.models as models

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from torchvision import transforms

In [ ]:
from PIL import Image
import glob, os

In [ ]:
from torchsummary import summary

In [ ]:
dirs = os.listdir("../input/flickr8k/Images")

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
device = "cpu"

In [ ]:
len(dirs)

In [ ]:
dirs.sort()

In [ ]:
dirs[0]

In [ ]:
PATH = "../input/flickr8k/Images/"
Image.open(PATH+dirs[0])

In [ ]:
cap = open('../input/flickr8k/captions.txt','r')
lines = cap.readlines()

In [ ]:
lines[6]

In [ ]:
len(sorted(set(dirs)))

In [ ]:
# model = models.vgg19(pretrained=True)

In [ ]:
mobilnet =  models.mobilenet_v2(pretrained=True).to(device)

In [ ]:
# summary(mobilnet, (3, 112, 112))

In [ ]:
img = Image.open(PATH+dirs[0])

In [ ]:
arr = np.asarray(img)

In [ ]:
arr.shape

In [ ]:
class FeatureExtractor():
    def __init__(self, device='cpu', pooling=False, dtype=torch.float32):
        self.preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((112,112)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.226, 0.225]),
        ])
        self.device, self.dtype = device, dtype
        self.mobilenet = models.mobilenet_v2(pretrained=True).to(device)
        self.mobilenet = nn.Sequential(*list(self.mobilenet.children())[:-1])
        if pooling:
            self.mobilenet.add_module('LastAvgPool', nn.AvgPool2d(4,4))
        self.mobilenet.eval()
    
    def extract_features(self, img):
        num_img = img.shape[0]
        img_pre = []
        img = img / 255.0
        for i in range(num_img):
            img_pre.append(self.preprocess(img[i].dtype))
        img_pre = torch.stack(img_pre).to(device)
        with torch.no_grad():
#             feat = 50
#             process_batch = 32
            feat = self.mobilenet(img_pre).squeeze(dim=0)
        return feat
        

In [ ]:
fe = FeatureExtractor(device=device)

## Preprocess

In [ ]:
preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((112,112)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.226, 0.225]),
        ])

In [ ]:
model = models.mobilenet_v2(pretrained=True)

In [ ]:
model.to("cpu")

In [ ]:
summary(model, (3, 112, 112))

In [ ]:
net = nn.Sequential(*list(model.children())[:-1])

In [ ]:
net.add_module('LastAvgPool', nn.AvgPool2d(4, 4))

In [ ]:
net.to(device)

In [ ]:
list(net.children())

In [ ]:
net.eval()

In [ ]:
def check(img_path):
    img = Image.open(PATH+img_path)
    arr = np.asarray(img)
    arr = arr/255.0
    arr = preprocess(arr).type(torch.float32).to(device).unsqueeze(dim=0)
    preds = net(arr)
    return preds

In [ ]:
dirs[0]

In [ ]:
preds = check(dirs[0])

In [ ]:
preds[0]

In [ ]:
preds.squeeze().shape

In [ ]:
img2feat = defaultdict(list)

In [ ]:
for i, img_path in tqdm(enumerate(dirs)):
    with torch.no_grad():
        img = Image.open(PATH+img_path)
        arr = np.asarray(img)
        arr = arr/255.0
        arr = preprocess(arr).type(torch.float32).to(device).unsqueeze(dim=0)
        preds = net(arr)
        img2feat[img_path] = preds.squeeze()
#         break

In [ ]:
import pickle

In [ ]:
with open('img2feat_flatv2.p','wb') as fp:
    pickle.dump(img2feat, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
img2feat[dirs[2]]

In [ ]:
dirs[0]

In [ ]:
with open('../input/imgflatv2/img2feat_flatv2.p','rb') as f:
    img2feat2 = pickle.load(f)

In [ ]:
img2feat2[dirs[1]]

In [ ]:
# mobilnet.device

In [ ]:
summary(mobilnet, (3,112,112))

In [ ]:
arr = torch.tensor(arr, dtype=torch.float32)

In [ ]:
tr = transforms.Resize((224,224))

In [ ]:
fe.preprocess(arr)

In [ ]:
preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((112,112)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.226, 0.225]),
        ])

In [ ]:
dtype = torch.float32

In [ ]:
net = nn.Sequential(*list(mobilnet.children())[:-1])

In [ ]:
ten.shape

In [ ]:
def plotTensor(ten):
    nparray = ten.cpu().clone().numpy()
    plt.imshow(np.transpose(nparray, (1,2,0)))

In [ ]:
plotTensor(ten)

In [ ]:
# ten.unsqueeze_(dim=0)
net(ten).shape

In [ ]:
preds = net(ten)

In [ ]:
plotTensor(preds[0][0])

In [ ]:
pp = preds[0][0].detach().clone()